In [102]:
from dotenv import load_dotenv
import os
import time

from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [103]:
load_dotenv(dotenv_path = r'C:\Users\User\Desktop\Project LLMs\RAG-LangChain\data\.env')
nvidia_api_key = os.getenv('NVIDIA_API_KEY')
typhoon_ai_api_key = os.getenv('TYPHOON_AI_API_KEY')
langsmith_api_key = os.getenv('LANGSMITH_API_KEY')
cohere_api_key = os.getenv('COHERE_API_KEY')

In [104]:
os.environ['LANGCHAIN_API_KEY'] = langsmith_api_key
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

In [105]:
with open(r'C:\Users\User\Desktop\Project LLMs\RAG-LangChain\Evaluation\test_question.txt', 'r', encoding="utf-8") as file:
    test_questions = file.readlines()

test_questions = [question.strip() for question in test_questions]
len(test_questions)

40

In [106]:
cohere_embedding_model = CohereEmbeddings(model="embed-multilingual-v3.0",
                                           cohere_api_key = cohere_api_key)
bge_m3_embedding_model = OllamaEmbeddings(model="bge-m3:latest")

In [107]:
cohere_vectorstore = FAISS.load_local(r"C:\Users\User\Desktop\Project LLMs\RAG-LangChain\data\vectorstore\faiss_index_cohere", cohere_embedding_model, allow_dangerous_deserialization = True)
bge_m3_vectorstore = FAISS.load_local(r"C:\Users\User\Desktop\Project LLMs\RAG-LangChain\data\vectorstore\faiss_index_bge_m3", bge_m3_embedding_model, allow_dangerous_deserialization = True)

In [108]:
nvidiallama33_70 = ChatNVIDIA(model="meta/llama-3.3-70b-instruct",
                                api_key = nvidia_api_key)

openaityphoon2_70 = ChatOpenAI(base_url = 'https://api.opentyphoon.ai/v1',
                        model = 'typhoon-v2-70b-instruct',
                        api_key = typhoon_ai_api_key)

In [109]:
vectorstores = {'cohere': cohere_vectorstore,
                'bge_m3': bge_m3_vectorstore}

llms = {'llama33_70': nvidiallama33_70,
        'phoon2_70': openaityphoon2_70}

projects = ['Cohere - LLaMA3.3 70B', 'Cohere - TYPHOON AI 2 70B', 'BGE M3 - LLaMA3.3 70B', 'BGE M3 - TYPHOON AI 2 70B']

In [110]:
def build_retriever_chain(model, vectorstore):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are Junior, a friendly and knowledgeable male AI assistant.\nYour role is to assist users by providing academic advice, explanations, and helpful insights based on the given context.\nAlways ensure your responses are clear, concise, and easy to understand.\nUse only the provided context to answer questions, and do not rely on external knowledge.\nIf the context lacks sufficient information, politely ask the user for clarification.\nContext: {context}"),
        ('human', "{input}")
    ])

    chain = create_stuff_documents_chain(
        llm = model,
        prompt = prompt
    )

    retriever = vectorstore.as_retriever()

    retriever_chain = create_retrieval_chain(
        retriever = retriever,
        combine_docs_chain = chain
    )

    return retriever_chain


In [ ]:
project_index = 0
for vector in vectorstores:
    vectorstore = vectorstores[vector]
    print(f'Vector Name: {vector}')

    for model in llms:
        llm = llms[model]
        print(f'Model Name: {model}')
        retriever_chain = build_retriever_chain(llm, vectorstore)
        os.environ['LANGCHAIN_PROJECT'] = projects[project_index]
        for i, question in enumerate(test_questions):
            results = retriever_chain.invoke({'input': question})
            print(f'Question-Answering Test {i+1}:')
            print(f'  Quesion: {results['input']}')
            print(f'  Answer: {results['answer']}')
            print('-----------------------------------------------')

        project_index += 1
        print('\n#==================================#\n')

    print('\n#==================================#\n')

Vector Name: cohere
Model Name: llama33_70
Question-Answering Test 1:
  Quesion: วิชาที่เปิดสอนสำหรับภาควิชาเคมีชั้่นปีที่ 1 ปีการศึกษา 2567 เทอม 1 มาสัก 1 วิชา
  Answer: วิชาเคมีทั่วไป 1 หรือ GENERAL CHEMISTRY I รหัสวิชา 513101-2560 เป็นวิชาที่เปิดสอนสำหรับภาควิชาเคมี ชั้นปีที่ 1 ปีการศึกษา 2567 เทอม 1
-----------------------------------------------
Question-Answering Test 2:
  Quesion: ช่วยแนะนำวิชาสำหรับภาควิชาคณิตศาสตร์หน่อยสัก 1 วิชา
  Answer: สำหรับภาควิชาคณิตศาสตร์หนึ่งในวิชาที่น่าสนใจและเหมาะแก่การเรียนรู้ คือ "เรื่องคัดเฉพาะทางคณิตศาสตร์ 1" หรือ "SELECTED TOPICS IN MATHEMATICS I" รหัสวิชา 511481-2560 จำนวนหน่วยกิต 3

วิชานี้เป็นวิชาบังคับของสาขาคณิตปี 4-8 และไม่มีเงื่อนไขการลงทะเบียนเรียน มีการเรียนการสอนในวันพฤหัสบดี เวลา 13:00-15:40 น. ในห้อง 4320 อาคารวิทย์ 4 โดยมีศาสตราจารย์ ดร.ไพโรจน์ สัตยธรรม เป็นอาจารย์ผู้สอน

วิชานี้มีรายละเอียดการเรียนการสอนเกี่ยวกับเรื่องที่น่าสนใจทางคณิตศาสตร์ ซึ่งสามารถช่วยเสริมสร้างความรู้และทักษะทางคณิตศาสตร์ของนักเรียนได้อย่างดี
----------------

RateLimitError: Error code: 429 - {'detail': "You've hit the rate limit for this service. For higher request limits, consider using the Typhoon model at https://api.together.ai/models."}